# 模型训练流程
* 1.Data provider 
    * a.Image data
    * b.random vector  
* 2.Build Compute graph  
    * a.generator 
    * b.discriminator 
    * c.DCGAN class 
* 3.training process 

In [2]:
import os
import tensorflow as tf
from tensorflow import logging
from tensorflow import gfile
import sys
import pprint
import numpy as np
import pickle
import random
import math
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)# 加载数据
output_dir = './local_run' # 输出文件夹路径
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir) # 创建输出文件夹

def get_default_params():
    return tf.contrib.training.HParams(
        z_dim = 100, # 随机噪声数据维度
        init_conv_size = 4, # 将输入噪声变化成feature_map时,其初始size为4x4
        g_channels = [128, 64, 32, 1],# 生成器中各反卷积层的通道数目
        d_channels = [32, 64, 128, 256], # 判别器中各卷积层的通道数据,各个卷积层size减半,然后通道数加半,然后pooling
        batch_size = 128, # 每批数量
        learning_rate = 0.002, 
        beta1 = 0.5, # adam参数
        img_size = 32, # 生成图像为32x32
    )
hparams = get_default_params()


Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# data provider
class MnistData(object):
    # z_dim随机向量的长度, img_size,将train中的img resize成img_size
    def __init__(self, mnist_train, z_dim, img_size):
        self._data = mnist_train
        self._example_num = len(self._data)
        # 随机向量使用正态分布生成
        self._z_data = np.random.standard_normal(self._example_num, z_dim) 
        # next batch 指示器
        self._indicator = 0
        # resize 图像大小,参数为要resize到的图像大小
        self._resize_mnist_img(img_size)
        # shuffle函数
        self._random_shuffle()
        
    def _random_shuffle(self):
        x = np.random.permutation(self._example_num)
        # 随机化train和z
        self._data = self._data[x]
        self._z_data = self._z_data[x]  
    
    def _resize_mnist_img(self, img_size):
        """resize the img to target imgsize,first numpy to PIL img"""
        # 1.在mnist中图像都被归一化,所以先将图像灰度级恢复到0~255,像素值类型为整形
        data = np.asarray(self._data * 255, np.uint8)
        # 2. 像素数据 [example_num, 784] -> [example_num, 28, 28]
        data = np.reshape((self._example_num, 28, 28))
        # 3.将每幅图像插值成img_size * img_size
        new_data = []
        for i in range(self._example_num):
            img = data[i]
            # 将numpy数据变成PIL对象
            img = Image.fromarray(img)
            img = img.reshape((img_size, img_size))
            # 再将PTL对象变成numpy对象
            img = np.asarray(img)
            # 判别器神经网络需要图像通道信息
            img = np.reshape((img_size,img_size, 1))
            new_data.append(img)
        # 将resize后的全部图片转成numpy类型,new_data这里还是一个字典
        new_data = np.asarray(new_data, dtype=np.float32)
        # 将new_data归一化,这里将像素数据归一化到-1~1直接,为了配合generator,G中使用的是tanH
        new_data = new_data / 127.5 - 1 
        # self._data [example_num, img_size, img_size ,1]
        self._data = new_data
        
        
    